# Advanced Querying Mongo

Importing libraries and setting up connection

In [2]:
from pymongo import MongoClient
from pymongo import ASCENDING, DESCENDING
import re

dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/companies


### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [3]:
# Your Code
list(db.companies.find({"name":"Babelgum"},{"name":1}))

[{'_id': ObjectId('52cdef7c4bab8bd675297da0'), 'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [4]:
# Your Code
list(db.companies.find({"number_of_employees":{ "$gt": 5000 }},{"_id":0,"name":1}).sort(
    [("number_of_employees", ASCENDING)]
).limit(20))


[{'name': 'Nintendo'},
 {'name': 'Hexaware Technologies'},
 {'name': 'Facebook'},
 {'name': 'OpenText'},
 {'name': 'LSI'},
 {'name': 'CPM Braxis'},
 {'name': 'Microchip Technologies'},
 {'name': 'Mediaset'},
 {'name': 'Mindray Medical International'},
 {'name': 'Baidu'},
 {'name': 'Dentsu'},
 {'name': 'Tata Communications'},
 {'name': 'Atmel'},
 {'name': 'Acxiom'},
 {'name': 'Acxiom'},
 {'name': 'SRA International'},
 {'name': "Moody's"},
 {'name': 'ManTech'},
 {'name': 'Adobe Systems'},
 {'name': 'Broadcom'}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [5]:
# Your Code
list(db.companies.find({"founded_year":{ "$in": list(range(2000,2005)) }},
                       {"_id":0,"name":1,"founded_year":1}).limit(10))

[{'name': 'Digg', 'founded_year': 2004},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'SmugMug', 'founded_year': 2002}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [6]:
# Your Code
# https://docs.mongodb.com/manual/tutorial/query-embedded-documents/
list(db.companies.find({"$and":[{"founded_year":{ "$lt": 2010 }},
                                {"ipo.valuation_amount": { "$gt": 100000000 }}]},
                       {"_id":0,"name":1,"ipo":1}).limit(3))

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [7]:
# Your Code
list(db.companies.find({"$and":[{"number_of_employees":{ "$lt": 1000 }},
                                {"founded_year": { "$lt": 2005 }}]},
                       {"_id":0,"name":1}).sort(
    [("number_of_employees", ASCENDING)]
).limit(10))

[{'name': 'Fox Interactive Media'},
 {'name': 'Skype'},
 {'name': 'Ticketmaster'},
 {'name': 'stylediary'},
 {'name': 'MindTouch'},
 {'name': 'Simpy'},
 {'name': 'Eurekster'},
 {'name': 'Compete'},
 {'name': 'EditGrid'},
 {'name': 'Monster'}]

### 6. All the companies that don't include the `partners` field.

In [16]:
# Your Code
list(db.companies.find({ "partners": { "$exists": True }},{"_id":0,"name":1}).limit(5))

[{'name': 'AdventNet'},
 {'name': 'Wetpaint'},
 {'name': 'Zoho'},
 {'name': 'Omnidrive'},
 {'name': 'Postini'}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [9]:
# Your Code
list(db.companies.find({ "category_code": { "$type": "null" }},{"_id":0,"name":1}).limit(5))

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [10]:
# Your Code
list(db.companies.find({"number_of_employees":{ "$in": list(range(100,999)) }},
                       {"_id":0,"name":1,"number_of_employees":1}).limit(5))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

### 9. Order all the companies by their IPO price in a descending order.

In [18]:
# Your Code
list(db.companies.find({ "_id": { "$exists": True }},{"_id":0,"name":1})
     .sort([("ipo.valuation_amount", DESCENDING)]).limit(10))

[{'name': 'GREE'},
 {'name': 'Facebook'},
 {'name': 'Amazon'},
 {'name': 'Twitter'},
 {'name': 'Groupon'},
 {'name': 'Tencent'},
 {'name': 'Western Digital'},
 {'name': 'LinkedIn'},
 {'name': 'BMC Software'},
 {'name': 'Rackspace'}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [19]:
# Your Code
list(db.companies.find({ "_id": { "$exists": True }},{"_id":0,"name":1,"number_of_employees":1})
     .sort([("number_of_employees", DESCENDING)]).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [21]:
# Your Code
list(db.companies.find({"founded_month":{ "$in": list(range(7,12)) }},
                       {"_id":0,"name":1,"founded_month":1}).limit(1000))

SyntaxError: invalid syntax (<ipython-input-21-b50ec5bac306>, line 3)

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [ ]:
# Your Code
list(db.companies.find({"$and":[{"founded_year":{ "$lt": 2000 }},
                                {"acquisitions": "valuation_amount": { "$gt": 1000000 }}]},
                       {"_id":0,"name":1,}).limit(3))

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
# Your Code

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code